# Generate the data for the ISDM-24 track

In [ ]:
%pip install -q pandas openpyxl jinja2

In [ ]:
# Print formatted dates in spanish locale
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

In [ ]:
# Read the dataframe from the .xlsx file given by Easychair
import pandas as pd
df = pd.read_excel('../info/ISDM2024.xlsx',index_col=0,header=0)

# df.Author to spanish 'and' -> 'y'
df['Authors'] = df['Authors'].str.replace(' and ', ' y ', regex=False)
df

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class Session:
    name: str
    start: datetime
    end: datetime
    location: str
    chair: str

isdm1 = Session('ISDM-1',
                datetime(2024,7,18,9,15),
                datetime(2024,7,18,10,15),
                'Aula A.3.1',
                'Diego Sevilla')

isdm2 = Session('ISDM-2',
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,17,30),
                'Aula A.3.1',
                'Pedro J. Clemente')

isdm3 = Session('ISDM-3',
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,19,00),
                'Aula A.3.1',
                'Diego Sevilla y Pedro J. Clemente')

sessions = {s.name : s for s in [isdm1, isdm2, isdm3]}

session_map = {isdm1.name: [91, 94, 24],
               isdm2.name: [3, 2, 4, 121, 124],
               isdm3.name: [12, 57, 81]}

assert set(session_map.keys()) == set(sessions.keys())
# No article left behind
assert set(df.index) == set().union(*session_map.values())

In [ ]:
# Produce the slide index from the slideindex template
from jinja2 import Template

def gen_session_index(sessions: dict[str, Session]) -> None:
    template = Template(open('templates/sessionindex.tex.j2').read())
    with open('../pr/sessionindex.tex', 'w') as f:
        f.write(template.render(sessions=sessions))

gen_session_index(sessions)

In [ ]:
def gen_session_slides(sessions: dict[str, Session], session_map: dict[str, list[int]], df) -> None:
    template = Template(open('templates/sessionslide.tex.j2').read())

    def get_duration(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return '20 minutos'
            case 'corto' | 'herramienta':
                return '10 minutos'
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def startdate_str(d: datetime) -> str:
        return d.strftime('%A, %d de %B de %Y, %H:%M')

    for session, papers in session_map.items():
        with open(f'../pr/{session}.tex', 'w') as f:
            f.write(template.render(session=sessions[session],
                                    papers=papers,
                                    startdate_str = startdate_str,
                                    articlestartdate = sessions[session].start + timedelta(minutes=10),
                                    df=df,
                                    get_duration=get_duration))

gen_session_slides(sessions, session_map, df)